In [1]:
import numpy as np
from scipy.sparse import csc_matrix, csr_matrix, load_npz, save_npz, hstack
from scipy.io import wavfile
from pydub import AudioSegment
import pandas as pd
from chord_scripts import *

# Тестовая версия

## Загрузка данных

In [2]:
full_spectr = load_npz('Testing spectrosplit/sparse.npz')

In [3]:
durat = AudioSegment.from_wav('Testing spectrosplit/audio.wav').duration_seconds
#durat = wav.duration_seconds
durat

148.06204081632654

In [4]:
def size_by_shape(shape):
    return shape[0]*shape[1]

In [5]:
full_spectr.shape

(1025, 51025)

In [6]:
spectr_block_time = durat/max(full_spectr.shape)
spectr_block_time

0.0029017548420642147

In [7]:
song_df = pd.read_csv('Testing spectrosplit/full.lab', delimiter='\t',header=None)
song_df.columns = ['start','stop','label']
song_df.head()

,start,stop,label
0,0.000000,0.139320,N
1,0.139320,1.895306,N
2,1.895306,3.651293,N
3,3.651293,5.385544,E:maj
4,5.385544,7.119796,C#:min


## Обработка данных

In [8]:
song_df['label'] = map_chords(song_df['label'])

In [9]:
song_df.head()

,start,stop,label
0,0.000000,0.139320,N
1,0.139320,1.895306,N
2,1.895306,3.651293,N
3,3.651293,5.385544,E:maj
4,5.385544,7.119796,C#:min


In [10]:
0.139320/spectr_block_time

48.01232618979357

In [25]:
chords_spectr = {}
for labeled_row in song_df.iterrows():
    idx, labeled_row = labeled_row
    start, stop = map(lambda x: int(x/spectr_block_time), [labeled_row['start'],labeled_row['stop']])
    if labeled_row['label'] in chords_spectr:    
        chords_spectr[labeled_row['label']]= hstack([full_spectr[:,start:stop],chords_spectr[labeled_row['label']]])
    else:
        chords_spectr[labeled_row['label']] = full_spectr[:,start:stop]

In [14]:
chords_spectr

{'A:maj': <1025x10643 sparse matrix of type '<type 'numpy.complex128'>'
 	with 78674 stored elements in Compressed Sparse Column format>,
 'B:maj': <1025x10643 sparse matrix of type '<type 'numpy.complex128'>'
 	with 81030 stored elements in Compressed Sparse Column format>,
 'C#:min': <1025x10645 sparse matrix of type '<type 'numpy.complex128'>'
 	with 102019 stored elements in Compressed Sparse Column format>,
 'E:maj': <1025x16631 sparse matrix of type '<type 'numpy.complex128'>'
 	with 113568 stored elements in Compressed Sparse Column format>,
 'N': <1025x2336 sparse matrix of type '<type 'numpy.complex128'>'
 	with 1089 stored elements in Compressed Sparse Column format>}

In [ ]:
plot_spectrogram(chords_spectr['N'].toarray())

In [ ]:
plot_spectrogram((chords_spectr['A:maj']-chords_spectr['B:maj']).toarray())

In [ ]:
plot_spectrogram(chords_spectr['A:maj'].toarray())

In [ ]:
plot_spectrogram(chords_spectr['B:maj'].toarray())

In [ ]:
plot_spectrogram(chords_spectr['C#:min'].toarray())

# Для всех файлов

In [29]:
all_songs = pd.read_csv('McGill-Billboard (2)/songs_converted.csv', index_col=0)
converted_songs = all_songs[all_songs['converted']==True]
print 'All songs:',all_songs.shape[0], 'converted:',converted_songs.shape[0]
converted_songs.head()

All songs: 739 converted: 442


,title,artist,number,duration,t_eps,href,status,converted
0,Chicago,25 or 6 to 4,176,4:50,0.0,https://ipleer.fm/song/101272384/Chicago_Chica...,Downloaded,True
1,Sukiyaki,A Taste Of Honey,1286,3:43,1.0,https://ipleer.fm/song/2787937/A_Taste_Of_Hone...,Downloaded,True
2,Chiquitita,ABBA,183,5:25,0.0,https://ipleer.fm/song/1958012/ABBA_-_Chiquitita/,Downloaded,True
3,"Knowing Me, Knowing You",ABBA,231,4:04,1.0,https://ipleer.fm/song/1823039/ABBA_-_Knowing_...,Downloaded,True
4,Honey Honey,ABBA,902,2:54,0.0,https://ipleer.fm/song/7201410/ABBA_-_Honey_Ho...,Downloaded,True


In [28]:
for song in all_songs.iterrows():
    song_idx, song = song
    
    dname = 'audio/billboard/'+'0'*(4-len(str(song.number)))+str(song.number)+'/'
    fname = format_fname('{}-{}.mp3'.format(song.artist,song.title))
    
    full_spectr = load_npz(dname+'sparse.npz')
    durat = AudioSegment.from_wav(dname+'audio.wav').duration_seconds
    spectr_block_time = durat/max(full_spectr.shape)
    
    chords_file_name = 'McGill-Billboard Parsed/'+'0'*(4-len(str(song.number)))+str(song.number)+'/full.lab'
    song_df = pd.read_csv(chords_file_name, delimiter='\t',header=None)
    song_df.columns = ['start','stop','label']
    
    chords_spectr = {}
    for labeled_row in song_df.iterrows():
        idx, labeled_row = labeled_row
        start, stop = map(lambda x: int(x/spectr_block_time), [labeled_row['start'],labeled_row['stop']])
        if labeled_row['label'] in chords_spectr:    
            chords_spectr[labeled_row['label']]= hstack([full_spectr[:,start:stop],chords_spectr[labeled_row['label']]])
        else:
            chords_spectr[labeled_row['label']] = full_spectr[:,start:stop]
    
    for chName, chSpectr in chords_spectr.items():
        try:
            !mkdir chords/{chName}
        except:
            pass
        save_npz('chords/'+chName,chSpectr)